In [ ]:
from functions import *

import numpy as np
import xarray as xr

import holoviews as hv
import geoviews as gv
from geoviews import opts
from cartopy import crs

gv.extension('bokeh', 'matplotlib')


%load_ext autoreload
%autoreload 2

In [ ]:
DATA_PATH = '/home/helvecioneto/disco1/dados/goes16/mmh/'
DATA_NAME = 'goes16'
DATA_PATTERN = '%%YYYY%%mm%%dd_%%HH%%MM.nc'
TRACK_PATH = '/home/helvecioneto/disco1/tracks/goes16mmh/track/'
FORECAST_PATH = '/home/helvecioneto/disco1/tracks/goes16mmh/forecast/'
TIME_DELTA = '10min'
LAST_IMAGES = 5
LAT_MIN = -56
LAT_MAX = 35
LON_MIN = -116
LON_MAX = -25
X_DIM = 910
Y_DIM = 910

In [ ]:
# PATHS
timestamp = get_latest_timestamp(TRACK_PATH)

pth_trck_bound, pth_trck_vec, pth_trck_trj = get_geotrack_paths(timestamp,TRACK_PATH)
pth_fcst_bound, pth_fcst_vec, pth_fcst_trj = get_geofrcst_paths(timestamp, 1, TIME_DELTA, FORECAST_PATH, )


trck_bound = read_geoframe(pth_trck_bound)
trck_vec = read_geoframe(pth_trck_vec)
trck_trj = read_geoframe(pth_trck_trj)
fcst_bound = read_geoframe(pth_fcst_bound)
fcst_vec = read_geoframe(pth_fcst_vec)
fcst_trj = read_geoframe(pth_fcst_trj)

In [ ]:
coords_array = True

data_array = np.full((LAST_IMAGES, X_DIM, Y_DIM),np.nan, dtype=np.float32)
timestamp_list = []

for i in range(LAST_IMAGES, 0, -1):
    path_stamp = timestamp - pd.Timedelta(TIME_DELTA * (i))
    data_path = get_data_paths(path_stamp, DATA_PATH+path_stamp.strftime('%Y/%m/%d/'), DATA_PATTERN)
    read_func = read_data(DATA_NAME)

    try:
        data = read_func(data_path)
    except:
        data = np.full((X_DIM, Y_DIM),np.nan, dtype=np.float32)

    if coords_array:
        lat = np.linspace(LAT_MIN, LAT_MAX, X_DIM)
        lon = np.linspace(LON_MIN, LON_MAX, Y_DIM)
        # lat = xr.DataArray(lat, dims=('lat'))
        # lon = xr.DataArray(lon, dims=('lon'))
        coords_array = False

    # Op in data
    data = data * 100
    data[data == 0] = np.nan
    # Add to array
    data_array[LAST_IMAGES-i] = data
    # Add to timestamp list
    timestamp_list.append(path_stamp)


# Create a xr.DataArray with the data, timestamps, lat and lon
data_xarray = xr.DataArray(data_array, coords=[timestamp_list, lat, lon], dims=['time', 'lat', 'lon'])
data_xarray.name = 'data'

kdims = ['time','lon', 'lat']
vdims = ['data']

# Create a HoloViews Dataset
data_set = gv.Dataset(data_xarray, kdims=kdims, vdims=vdims)


In [ ]:
# Create a gv image
threshold_image = data_set.to(gv.Image, ['lon', 'lat'])

In [ ]:
background = hv.element.tiles.StamenTerrainRetina()

In [ ]:
imag_plot = background * threshold_image

In [ ]:
imag_plot = imag_plot.opts(
    opts.Image(cmap='turbo', colorbar=True, tools=['hover'], width=800, height=600),
    opts.Overlay()
)

In [ ]:
imag_plot

In [ ]:
# save to html
hv.save(imag_plot, '/home/helvecioneto/plot.html')

In [ ]:
# # Create an xarray dataseta

# data = xr.DataArray(data, coords=[lat, lon], dims=['lat', 'lon'])
# data.name = 'data'
# data.attrs['crs'] = 'EPSG:4326'
# data.attrs['extent'] = (LON_MIN, LON_MAX, LAT_MIN, LAT_MAX)
# dataset = gv.Dataset(data, crs=crs.PlateCarree(), kdims=['lon', 'lat'], vdims=['data'])

In [ ]:
# threhold_image = gv.Image(dataset, ['lon', 'lat'], ['data'])
# background = hv.element.tiles.StamenTerrainRetina()

In [ ]:
# Save html
hv.save(imag_plot, '/var/www/html/helvecioneto.github.io/trackboard/index.html', backend='bokeh')